In [1]:
import pandas as pd
import numpy as np
from statsmodels.sandbox.stats.runs import runstest_1samp

# Função para testar cada sequência
def testa_sequencia(seq_str):
    seq_binaria = np.array(list(seq_str), dtype=int)
    return runstest_1samp(seq_binaria, cutoff='mean', correction=True)

# Caminho do arquivo CSV com as sequências binárias
file_path = "3-seqbinaria_participantes.csv"
output_path = "resultado_teste_corridas_participantes.csv"

try:
    df = pd.read_csv(file_path, sep="\t", header=None, names=['Candidato', 'Mediana', 'Sequencia'])

    results = []

    for _, row in df.iterrows():
        seq_str = row['Sequencia']
        seq_binaria = np.array(list(seq_str), dtype=int)
        n_1s = np.sum(seq_binaria)
        n_0s = len(seq_binaria) - n_1s
        stat, p_val = runstest_1samp(seq_binaria, cutoff='mean', correction=True)

        if p_val > 0.05:
            interpretacao = "Aceitaram H₀"
        elif p_val > 0.01:
            interpretacao = "Rejeitaram H₀ moderadamente"
        else:
            interpretacao = "Rejeitaram H₀ fortemente"

        results.append({
            'candidato': row['Candidato'],
            'mediana': row['Mediana'],
            'tamanho_sequencia': len(seq_binaria),
            'n_1s': n_1s,
            'n_0s': n_0s,
            'p_valor': round(p_val, 6),
            'interpretacao': interpretacao
        })

    df_results = pd.DataFrame(results)

    # Mapeamento da faixa de p-valor
    faixa_dict = {
        "Aceitaram H₀": "p > 0.05",
        "Rejeitaram H₀ moderadamente": "0.01 < p ≤ 0.05",
        "Rejeitaram H₀ fortemente": "p ≤ 0.01"
    }

    resumo = df_results['interpretacao'].value_counts().reset_index()
    resumo.columns = ['Resultado do Teste', 'Quantidade']
    resumo['Faixa de p-valor'] = resumo['Resultado do Teste'].map(faixa_dict)
    resumo['Percentual (%)'] = (resumo['Quantidade'] / len(df_results) * 100).round(2)

    # Reordenar colunas
    resumo = resumo[['Resultado do Teste', 'Faixa de p-valor', 'Quantidade', 'Percentual (%)']]

    # Reordenar linhas conforme lógica
    ordem = ["Aceitaram H₀", "Rejeitaram H₀ moderadamente", "Rejeitaram H₀ fortemente"]
    resumo['ordem'] = resumo['Resultado do Teste'].apply(lambda x: ordem.index(x))
    resumo = resumo.sort_values('ordem').drop(columns='ordem')

    # Exibir apenas o resumo
    print(resumo.to_string(index=False))

    # Salvar resultados completos
    df_results.to_csv(output_path, index=False)
    print()
    print()
    print(f"Arquivo '{output_path}' salvo com sucesso.")

except FileNotFoundError:
    print(f"Erro: o arquivo '{file_path}' não foi encontrado.")

except pd.errors.EmptyDataError:
    print(f"Erro: o arquivo '{file_path}' está vazio ou mal formatado.")

except Exception as e:
    print(f"Ocorreu um erro ao processar os dados: {e}")


         Resultado do Teste Faixa de p-valor  Quantidade  Percentual (%)
               Aceitaram H₀         p > 0.05          53           96.36
Rejeitaram H₀ moderadamente  0.01 < p ≤ 0.05           2            3.64


Arquivo 'resultado_teste_corridas_participantes.csv' salvo com sucesso.
